In [1]:
import sagemaker
import boto3
import pandas as pd
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri
 
region = boto3.Session().region_name
 
session = sagemaker.Session()
bucket = session.default_bucket()
print(bucket)
prefix = 'sagemaker-demo'

sm = boto3.Session().client(service_name='sagemaker',region_name=region)

sagemaker-us-east-1-029864677731
sagemaker-us-east-1-029864677731


In [2]:
!wget -N https://gist.githubusercontent.com/michhar/2dfd2de0d4f8727f873422c5d959fff5/raw/fa71405126017e6a37bea592440b4bee94bf7b9e/titanic.csv
local_data_path = 'titanic.csv'

--2020-07-25 18:54:02--  https://gist.githubusercontent.com/michhar/2dfd2de0d4f8727f873422c5d959fff5/raw/fa71405126017e6a37bea592440b4bee94bf7b9e/titanic.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 151.101.248.133
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|151.101.248.133|:443... connected.
--2020-07-25 18:54:02--  https://gist.githubusercontent.com/michhar/2dfd2de0d4f8727f873422c5d959fff5/raw/fa71405126017e6a37bea592440b4bee94bf7b9e/titanic.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 151.101.248.133
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|151.101.248.133|:443... connected.
HTTP request sent, awaiting response... HTTP request sent, awaiting response... 200 OK
Length: 60301 (59K) [text/plain]
Saving to: ‘titanic.csv’

titanic.csv         100%[===================>]  58.89K  --.-KB/s    in 0.002s  

Last-modified header missing -- time-stamps turned off.
2020-07-25 18:54:02 (

In [3]:
data = pd.read_csv(local_data_path)
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
data = data.drop(['Name', 'Ticket', 'Cabin', 'Embarked', 'PassengerId'], axis = 1)
data['Sex'] = data['Sex'].astype('category').cat.codes
data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,1,22.0,1,0,7.2500
1,1,1,0,38.0,1,0,71.2833
2,1,3,0,26.0,0,0,7.9250
3,1,1,0,35.0,1,0,53.1000
4,0,3,1,35.0,0,0,8.0500


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,1,22.0,1,0,7.2500
1,1,1,0,38.0,1,0,71.2833
2,1,3,0,26.0,0,0,7.9250
3,1,1,0,35.0,1,0,53.1000
4,0,3,1,35.0,0,0,8.0500


In [5]:
# XGBoost requires no headers
# Target is first column
train_data = data.sample(frac=0.8,random_state=200)
test_data = data.drop(train_data.index)
test_y = test_data['Survived']
test_data = test_data.drop(columns=['Survived'])

train_data = pd.DataFrame(train_data[:].values)
test_data = pd.DataFrame(test_data[:].values)

In [6]:
train_file = 'train_data.csv';
train_data.to_csv(train_file, index=False, header=True)
train_data_s3_path = session.upload_data(path=train_file, key_prefix=prefix)
print('Train dataset uploaded to: ' + train_data_s3_path)
 
test_file = 'test_data.csv';
test_data.to_csv(test_file, index=False, header=False)
test_data_s3_path = session.upload_data(path=test_file, key_prefix=prefix)
print('Test dataset uploaded to: ' + test_data_s3_path)

Train dataset uploaded to: s3://sagemaker-us-east-1-029864677731/sagemaker-demo/train_data.csv
Test dataset uploaded to: s3://sagemaker-us-east-1-029864677731/sagemaker-demo/test_data.csv
Train dataset uploaded to: s3://sagemaker-us-east-1-029864677731/sagemaker-demo/train_data.csv
Test dataset uploaded to: s3://sagemaker-us-east-1-029864677731/sagemaker-demo/test_data.csv


In [7]:
container = get_image_uri(session.boto_region_name, 'xgboost')
role = get_execution_role()

'get_image_uri' method will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
	get_image_uri(region, 'xgboost', '1.0-1').
'get_image_uri' method will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
	get_image_uri(region, 'xgboost', '1.0-1').


In [8]:
xgb = sagemaker.estimator.Estimator(container, # The location of the container we wish to use
                                    role,   # What is our current IAM Role
                                    train_instance_count=1, # How many compute instances
                                    train_instance_type='ml.m5.large', # What kind of compute instances
                                    output_path='s3://{}/{}/output'.format(session.default_bucket(), prefix),
                                    sagemaker_session=session)

In [9]:
xgb.set_hyperparameters(max_depth=2,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        silent=0,
                        objective='binary:logistic',
                        early_stopping_rounds=10,
                        num_round=500)

In [ ]:
s3_input_train = sagemaker.s3_input(s3_data=train_data_s3_path, content_type='csv')
s3_input_test = sagemaker.s3_input(s3_data=test_data_s3_path, content_type='csv')

xgb.fit({'train': s3_input_train})

2020-07-25 18:54:03 Starting - Starting the training job.2020-07-25 18:54:03 Starting - Starting the training job.....
2020-07-25 18:54:05 Starting - Launching requested ML instances.
2020-07-25 18:54:05 Starting - Launching requested ML instances.........

In [ ]:
# Batch transform

# reloaded_xgb_model = sagemaker.estimator.Estimator.attach(model_job_name)

xgb_transformer = xgb.transformer(instance_count = 1,
                                  instance_type = 'ml.m5.large',
                                  output_path='s3://{}/{}/out/'.format(bucket, prefix))
                                  
# Start the transform job
xgb_transformer.transform(test_data_s3_path,
                          content_type='text/csv', #pecify the content type 
                          split_type='Line') #split type of the test data


xgb_transformer.wait()

In [ ]:
predictions = pd.read_csv('s3://{}/{}/out/{}.out'.format(bucket, prefix, test_file), header=None)
predictions.head()

In [ ]:
xgb_predictor = xgb.deploy(initial_instance_count=1, instance_type='ml.m5.large')

In [ ]:
from sagemaker.predictor import csv_serializer
import numpy as np
xgb_predictor.content_type = 'text/csv'
xgb_predictor.serializer = csv_serializer

Y_pred = xgb_predictor.predict(test_data.values).decode('utf-8')
Y_pred = np.fromstring(Y_pred, sep=',')
print(Y_pred)

In [ ]:
xgb_predictor.delete_endpoint()